In [12]:
import time
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

# 讀取 CSV
csv_file = "all.csv"
df = pd.read_csv(csv_file, dtype=str)

# 初始化 WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # 無頭模式
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920x1080")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 爬取 Google Maps
base_url = "https://www.google.com.tw/maps/search/{}"

for index, row in df.iterrows():
    try:
        viewpoint = row['viewpoint']
        search_url = base_url.format(viewpoint)
        driver.get(search_url)
        time.sleep(3)
        
        # 解析搜尋結果
        results = driver.find_elements(By.CSS_SELECTOR, "div.Nv2PK")
        
        if len(results) == 0:
            # 無搜尋結果，跳過
            continue
        
        if len(results) == 1:
            # 直接進入地點頁面
            location_url = driver.current_url
            location_name = viewpoint
        else:
            # 多個搜尋結果，選擇最多評論的地點
            print(f"找到 {len(results)} 個搜尋結果，選擇評論數最多的景點")
            max_reviews = 0
            best_option = None
            
            for result in results:
                try:
                    review_span = result.find_element(By.CSS_SELECTOR, 'span.ZkP5Je')
                    aria_label = review_span.get_attribute("aria-label")
                    match = re.search(r'([\d,]+) 則評論', aria_label)
                    
                    if match:
                        reviews = int(match.group(1).replace(',', ''))
                        if reviews > max_reviews:
                            max_reviews = reviews
                            best_option = result.find_element(By.CSS_SELECTOR, 'a.hfpxzc')
                except:
                    continue
            
            if best_option:
                location_name = best_option.get_attribute("aria-label")
                location_url = best_option.get_attribute("href")
                driver.get(location_url)
                time.sleep(3)
            else:
                continue
        
        df.at[index, 'gmap_location'] = str(location_name)
        
        # 等待 URL 更新
        time.sleep(3)
        location_url = driver.current_url
        df.at[index, 'LOCATION_URL'] = str(location_url)
        
        # 取得 GPS 座標
        if "@" in location_url:
            gps_coords = location_url.split("@")[1].split(",17z")[0]
            df.at[index, 'GPS_coordinates'] = str(gps_coords)
        
        # 取得 1s 碼與 2s 碼
        try:
            # 解析 place_id
            regex1s01 = r'0x\w+'
            place_id_matches = re.findall(regex1s01, location_url)
            if len(place_id_matches) >= 2:
                place_id = place_id_matches[0] + "%3A" + place_id_matches[1]
            else:
                place_id = ""
            
            # 解析 KEI 值
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, "lxml")
            script_tags = soup.find_all("script")
            kei_value = ""
            for script in script_tags:
                if "kEI" in script.text:
                    start = script.text.find("kEI='") + len("kEI='")
                    end = script.text.find("'", start)
                    kei_value = script.text[start:end]
                    break
            
            if place_id and kei_value:
                rq_url = f"https://www.google.com.tw/maps/rpc/listugcposts?authuser=0&hl=zh-TW&gl=tw&pb=!1m6!1s{place_id}!6m4!4m1!1e1!4m1!1e3!2m2!1i10!2s!5m2!1s{kei_value}!7e81!8m9!2b1!3b1!5b1!7b1!12m4!1b1!2b1!4m1!1e1!11m0!13m1!1e2"
                df.at[index, 'RQ_URL'] = str(rq_url)
        except:
            df.at[index, 'RQ_URL'] = ""
        
        # 即時寫入 CSV
        df.to_csv(csv_file, index=False)
        
    except Exception as e:
        print(f"Error processing {viewpoint}: {e}")
        continue

driver.quit()


找到 4 個搜尋結果，選擇評論數最多的景點
找到 2 個搜尋結果，選擇評論數最多的景點
找到 3 個搜尋結果，選擇評論數最多的景點
找到 7 個搜尋結果，選擇評論數最多的景點
找到 7 個搜尋結果，選擇評論數最多的景點
找到 10 個搜尋結果，選擇評論數最多的景點
找到 4 個搜尋結果，選擇評論數最多的景點
找到 7 個搜尋結果，選擇評論數最多的景點
找到 4 個搜尋結果，選擇評論數最多的景點
找到 5 個搜尋結果，選擇評論數最多的景點
找到 3 個搜尋結果，選擇評論數最多的景點
找到 8 個搜尋結果，選擇評論數最多的景點
找到 10 個搜尋結果，選擇評論數最多的景點
找到 4 個搜尋結果，選擇評論數最多的景點
找到 7 個搜尋結果，選擇評論數最多的景點
找到 7 個搜尋結果，選擇評論數最多的景點
找到 2 個搜尋結果，選擇評論數最多的景點
找到 3 個搜尋結果，選擇評論數最多的景點
找到 8 個搜尋結果，選擇評論數最多的景點
找到 2 個搜尋結果，選擇評論數最多的景點
找到 4 個搜尋結果，選擇評論數最多的景點
找到 7 個搜尋結果，選擇評論數最多的景點
找到 10 個搜尋結果，選擇評論數最多的景點
找到 4 個搜尋結果，選擇評論數最多的景點
找到 7 個搜尋結果，選擇評論數最多的景點
找到 4 個搜尋結果，選擇評論數最多的景點
找到 10 個搜尋結果，選擇評論數最多的景點
找到 10 個搜尋結果，選擇評論數最多的景點
找到 4 個搜尋結果，選擇評論數最多的景點
找到 4 個搜尋結果，選擇評論數最多的景點
找到 5 個搜尋結果，選擇評論數最多的景點
找到 4 個搜尋結果，選擇評論數最多的景點
找到 4 個搜尋結果，選擇評論數最多的景點
找到 2 個搜尋結果，選擇評論數最多的景點
找到 3 個搜尋結果，選擇評論數最多的景點
找到 2 個搜尋結果，選擇評論數最多的景點
找到 4 個搜尋結果，選擇評論數最多的景點
找到 9 個搜尋結果，選擇評論數最多的景點
找到 3 個搜尋結果，選擇評論數最多的景點
找到 3 個搜尋結果，選擇評論數最多的景點
找到 10 個搜尋結果，選擇評論數最多的景點
找到 4 個搜尋結果，選擇評論數最多的景點
找到 3 個搜尋結果，選擇評論數最多的景點
找到 2 個搜尋結果，選擇評論數最多的景點
找到 4 個搜尋結果，選擇評論數最多的景點
找到 3

In [10]:
print(place_id)

%E5%9C%8B%E7%AB%8B%E6%95%85%E5%AE%AE%E5%8D%9A%E7%89%A9%E9%99%A2/@25.0984008,121.546462,17z/data=


In [11]:
print(kei_value)